In [ ]:
pip install pennylane torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 804.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import pennylane as qml
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Configurații
NUM_RUNS = 5  # Numărul de rulări
PREPROCESSED_FILE = "/content/drive/MyDrive/QuantumElearningDataSet/processed_summaries_q_s.csv"
MAIN_DATA_FILE = "/content/drive/MyDrive/QuantumElearningDataSet/arxiv_quantum_computing_pdf.csv"
OUTPUT_FILE = "/content/drive/MyDrive/Quantum-LogFiles/15_cos_similarity_results.csv"
LOG_FILE = "/content/drive/MyDrive/Quantum-LogFiles/15_cos_similarity_logs.txt"
N_QUBITS = 15  # Numărul de qubiți pentru circuitul cuantic

# Funcție pentru maparea caracteristicilor cuantice
def quantum_feature_map(x, run_id):
    n_qubits = len(x)
    dev = qml.device("default.qubit", wires=n_qubits)
    @qml.qnode(dev)
    def circuit():
        for i, feature in enumerate(x):
            qml.RY(feature, wires=i)
        weights = np.random.random(size=(1, n_qubits))
        qml.templates.BasicEntanglerLayers(weights=weights, wires=range(n_qubits))
        return qml.state()
    return circuit()

# Funcție pentru calculul similarității cuantice
def quantum_similarity(vectors, run_id):
    quantum_states = [quantum_feature_map(vec, run_id) for vec in vectors]
    quantum_real_parts = [np.real(state) for state in quantum_states]
    similarities = cosine_similarity(quantum_real_parts)
    return similarities

# Funcție pentru preprocesarea datelor
def preprocess_data():
    # Încărcare date preprocesate
    df_processed = pd.read_csv(PREPROCESSED_FILE)

    # Convertire text în vectori TF-IDF
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(df_processed["Processed_Summary"]).toarray()

    # Reducere dimensionalitate
    svd = TruncatedSVD(n_components=N_QUBITS)
    reduced_matrix = svd.fit_transform(tfidf_matrix)

    return reduced_matrix

# Funcție pentru rularea unei singure iterații
def run_single_iteration(run_id, df, reduced_matrix):
    start_time = time.time()

    # Generare matrice de similaritate cuantică
    similarity_matrix = quantum_similarity(reduced_matrix, run_id)

    metrics = {
        'run_id': run_id,
        'nr_diferente': 0,
        'total_corecte': 0,
        'perfect_matches': 0,
        'at_least_one_mismatch': 0,
        'accuracies': []
    }

    for i, row in df.iterrows():
        similarities = similarity_matrix[i]
        top_5_indices = np.argsort(similarities)[-6:-1][::-1]
        current_category = df.iloc[i]["Category"]
        nr_corecte = sum(df.iloc[idx]["Category"] == current_category for idx in top_5_indices)

        metrics['accuracies'].append((nr_corecte / 5) * 100)
        metrics['total_corecte'] += nr_corecte
        metrics['nr_diferente'] += (5 - nr_corecte)

        if nr_corecte == 5:
            metrics['perfect_matches'] += 1
        else:
            metrics['at_least_one_mismatch'] += 1

    # Calcul metrici finale
    total_predicted = len(df) * 5
    metrics.update({
        'correct_percentage': (metrics['total_corecte'] / total_predicted) * 100,
        'different_percentage': (metrics['nr_diferente'] / total_predicted) * 100,
        'perfect_match_percentage': (metrics['perfect_matches'] / len(df)) * 100,
        'mismatch_percentage': (metrics['at_least_one_mismatch'] / len(df)) * 100,
        'mean_accuracy': np.mean(metrics['accuracies']),
        'execution_time': time.time() - start_time
    })

    return metrics

# Pregătire fișier de log
with open(LOG_FILE, "w") as f:
    f.write(f"Quantum Similarity Experiment - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Configuration: {NUM_RUNS} runs, {N_QUBITS} qubits\n\n")

# Încărcare date principale
df_main = pd.read_csv(MAIN_DATA_FILE)
reduced_matrix = preprocess_data()
all_metrics = []

# Rulări multiple
for run in range(1, NUM_RUNS+1):
    print(f"\n{'='*40}\nRun {run}/{NUM_RUNS}\n{'='*40}")

    metrics = run_single_iteration(run, df_main, reduced_matrix)
    all_metrics.append(metrics)

    # Scriere log
    log_entry = f"""Run {run} Results:
✅ Correct: {metrics['total_corecte']} ({metrics['correct_percentage']:.2f}%)
❌ Wrong: {metrics['nr_diferente']} ({metrics['different_percentage']:.2f}%)
🎯 Perfect: {metrics['perfect_matches']} ({metrics['perfect_match_percentage']:.2f}%)
⚠️ Mismatch: {metrics['at_least_one_mismatch']} ({metrics['mismatch_percentage']:.2f}%)
📊 Accuracy: {metrics['mean_accuracy']:.2f}%
⏱️ Time: {metrics['execution_time']:.2f}s\n"""

    print(log_entry)
    with open(LOG_FILE, "a") as f:
        f.write(log_entry)

# Calcul medii
if NUM_RUNS > 1:
    avg_metrics = {
        'correct_percentage': np.mean([m['correct_percentage'] for m in all_metrics]),
        'different_percentage': np.mean([m['different_percentage'] for m in all_metrics]),
        'perfect_match_percentage': np.mean([m['perfect_match_percentage'] for m in all_metrics]),
        'mismatch_percentage': np.mean([m['mismatch_percentage'] for m in all_metrics]),
        'mean_accuracy': np.mean([m['mean_accuracy'] for m in all_metrics]),
        'execution_time': np.mean([m['execution_time'] for m in all_metrics])
    }

    summary_log = f"""\n{'='*40}
FINAL SUMMARY AFTER {NUM_RUNS} RUNS
Average correct: {avg_metrics['correct_percentage']:.2f}%
Average wrong: {avg_metrics['different_percentage']:.2f}%
Average perfect: {avg_metrics['perfect_match_percentage']:.2f}%
Average mismatch: {avg_metrics['mismatch_percentage']:.2f}%
Overall accuracy: {avg_metrics['mean_accuracy']:.2f}%
Average time: {avg_metrics['execution_time']:.2f}s\n"""

    print(summary_log)
    with open(LOG_FILE, "a") as f:
        f.write(summary_log)

# Salvare rezultate detaliate
results_df = pd.DataFrame(all_metrics)
# results_df.to_csv(OUTPUT_FILE, index=False)
# print(f"\n📊 Results saved to: {OUTPUT_FILE}")
print(f"📄 Full log saved to: {LOG_FILE}")


Run 1/5
Run 1 Results:
✅ Correct: 2359 (96.29%)
❌ Wrong: 91 (3.71%)
🎯 Perfect: 457 (93.27%)
⚠️ Mismatch: 33 (6.73%)
📊 Accuracy: 96.29%
⏱️ Time: 21.02s


Run 2/5
Run 2 Results:
✅ Correct: 2365 (96.53%)
❌ Wrong: 85 (3.47%)
🎯 Perfect: 459 (93.67%)
⚠️ Mismatch: 31 (6.33%)
📊 Accuracy: 96.53%
⏱️ Time: 22.45s


Run 3/5
Run 3 Results:
✅ Correct: 2366 (96.57%)
❌ Wrong: 84 (3.43%)
🎯 Perfect: 461 (94.08%)
⚠️ Mismatch: 29 (5.92%)
📊 Accuracy: 96.57%
⏱️ Time: 19.15s


Run 4/5
Run 4 Results:
✅ Correct: 2380 (97.14%)
❌ Wrong: 70 (2.86%)
🎯 Perfect: 455 (92.86%)
⚠️ Mismatch: 35 (7.14%)
📊 Accuracy: 97.14%
⏱️ Time: 22.63s


Run 5/5
Run 5 Results:
✅ Correct: 2364 (96.49%)
❌ Wrong: 86 (3.51%)
🎯 Perfect: 458 (93.47%)
⚠️ Mismatch: 32 (6.53%)
📊 Accuracy: 96.49%
⏱️ Time: 19.31s


FINAL SUMMARY AFTER 5 RUNS
Average correct: 96.60%
Average wrong: 3.40%
Average perfect: 93.47%
Average mismatch: 6.53%
Overall accuracy: 96.60%
Average time: 20.91s

📄 Full log saved to: /content/drive/MyDrive/Quantum-LogFiles/15_cos

METODA 2-QUANTUM FIDELITY

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import pennylane as qml
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Configurații
NUM_RUNS = 5  # Numărul de rulări
PREPROCESSED_FILE = "/content/drive/MyDrive/QuantumElearningDataSet/processed_summaries_q_s.csv"
MAIN_DATA_FILE = "/content/drive/MyDrive/QuantumElearningDataSet/arxiv_quantum_computing_pdf.csv"
LOG_FILE = "/content/drive/MyDrive/Quantum-LogFiles/8_quanutum_fidelity_logs.txt_v2"
N_QUBITS = 8  # Numărul de qubiți pentru circuitul cuantic

# Funcție pentru maparea caracteristicilor cuantice
def quantum_feature_map(x):
    n_qubits = len(x)
    dev = qml.device("default.qubit", wires=n_qubits)
    @qml.qnode(dev)
    def circuit():
        for i, feature in enumerate(x):
            qml.RY(feature, wires=i)
        weights = np.random.random(size=(1, n_qubits))
        qml.templates.BasicEntanglerLayers(weights=weights, wires=range(n_qubits))
        return qml.state()
    return circuit()

# Funcție pentru calculul similarității cuantice
def quantum_similarity(vectors):
    quantum_states = np.array([quantum_feature_map(vec) for vec in vectors])
    density_matrices = qml.math.dm_from_state_vector(quantum_states)  # Vectorizat

    # Calcul fidelitate pairwise folosind broadcasting
    n = len(quantum_states)
    fidelities = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):  # Triunghi superior
            fidelities[i][j] = qml.math.fidelity(density_matrices[i], density_matrices[j])
            fidelities[j][i] = fidelities[i][j]  # Simetrie
    return fidelities

# def quantum_similarity(vectors):
#     quantum_states = [quantum_feature_map(vec) for vec in vectors]
#     fidelities = np.zeros((len(quantum_states), len(quantum_states)))

#     for i in range(len(quantum_states)):
#         for j in range(len(quantum_states)):
#             if i<=j:
#               # Convert the quantum state vectors to density matrices
#               state_i = qml.math.dm_from_state_vector(quantum_states[i])
#               state_j = qml.math.dm_from_state_vector(quantum_states[j])

#               # Calculate the fidelity between the two states
#               fidelity = qml.math.fidelity(state_i, state_j)
#               fidelities[i][j] = fidelity
#               fidelities[j][i] = fidelity
#     return fidelities

# Funcție pentru preprocesarea datelor
def preprocess_data():
    # Încărcare date preprocesate
    df_processed = pd.read_csv(PREPROCESSED_FILE)

    # Convertire text în vectori TF-IDF
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(df_processed["Processed_Summary"]).toarray()

    # Reducere dimensionalitate
    svd = TruncatedSVD(n_components=N_QUBITS)
    reduced_matrix = svd.fit_transform(tfidf_matrix)

    return reduced_matrix

# Funcție pentru rularea unei singure iterații
def run_single_iteration(run_id, df, reduced_matrix):
    start_time = time.time()

    # Generare matrice de similaritate cuantică
    similarity_matrix = quantum_similarity(reduced_matrix)

    metrics = {
        'run_id': run_id,
        'nr_diferente': 0,
        'total_corecte': 0,
        'perfect_matches': 0,
        'at_least_one_mismatch': 0,
        'accuracies': []
    }

    for i, row in df.iterrows():
        similarities = similarity_matrix[i]
        top_5_indices = np.argsort(similarities)[-6:-1][::-1]
        current_category = df.iloc[i]["Category"]
        nr_corecte = sum(df.iloc[idx]["Category"] == current_category for idx in top_5_indices)

        metrics['accuracies'].append((nr_corecte / 5) * 100)
        metrics['total_corecte'] += nr_corecte
        metrics['nr_diferente'] += (5 - nr_corecte)

        if nr_corecte == 5:
            metrics['perfect_matches'] += 1
        else:
            metrics['at_least_one_mismatch'] += 1

    # Calcul metrici finale
    total_predicted = len(df) * 5
    metrics.update({
        'correct_percentage': (metrics['total_corecte'] / total_predicted) * 100,
        'different_percentage': (metrics['nr_diferente'] / total_predicted) * 100,
        'perfect_match_percentage': (metrics['perfect_matches'] / len(df)) * 100,
        'mismatch_percentage': (metrics['at_least_one_mismatch'] / len(df)) * 100,
        'mean_accuracy': np.mean(metrics['accuracies']),
        'execution_time': time.time() - start_time
    })

    return metrics

# Pregătire fișier de log
with open(LOG_FILE, "w") as f:
    f.write(f"Quantum Similarity Experiment - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Configuration: {NUM_RUNS} runs, {N_QUBITS} qubits\n\n")

# Încărcare date principale
df_main = pd.read_csv(MAIN_DATA_FILE)
reduced_matrix = preprocess_data()
all_metrics = []

# Rulări multiple
for run in range(1, NUM_RUNS+1):
    print(f"\n{'='*40}\nRun {run}/{NUM_RUNS}\n{'='*40}")

    metrics = run_single_iteration(run, df_main, reduced_matrix)
    all_metrics.append(metrics)

    # Scriere log
    log_entry = f"""Run {run} Results:
✅ Correct: {metrics['total_corecte']} ({metrics['correct_percentage']:.2f}%)
❌ Wrong: {metrics['nr_diferente']} ({metrics['different_percentage']:.2f}%)
🎯 Perfect: {metrics['perfect_matches']} ({metrics['perfect_match_percentage']:.2f}%)
⚠️ Mismatch: {metrics['at_least_one_mismatch']} ({metrics['mismatch_percentage']:.2f}%)
📊 Accuracy: {metrics['mean_accuracy']:.2f}%
⏱️ Time: {metrics['execution_time']:.2f}s\n"""

    print(log_entry)
    with open(LOG_FILE, "a") as f:
        f.write(log_entry)

# Calcul medii
if NUM_RUNS > 1:
    avg_metrics = {
        'correct_percentage': np.mean([m['correct_percentage'] for m in all_metrics]),
        'different_percentage': np.mean([m['different_percentage'] for m in all_metrics]),
        'perfect_match_percentage': np.mean([m['perfect_match_percentage'] for m in all_metrics]),
        'mismatch_percentage': np.mean([m['mismatch_percentage'] for m in all_metrics]),
        'mean_accuracy': np.mean([m['mean_accuracy'] for m in all_metrics]),
        'execution_time': np.mean([m['execution_time'] for m in all_metrics])
    }

    summary_log = f"""\n{'='*40}
FINAL SUMMARY AFTER {NUM_RUNS} RUNS
Average correct: {avg_metrics['correct_percentage']:.2f}%
Average wrong: {avg_metrics['different_percentage']:.2f}%
Average perfect: {avg_metrics['perfect_match_percentage']:.2f}%
Average mismatch: {avg_metrics['mismatch_percentage']:.2f}%
Overall accuracy: {avg_metrics['mean_accuracy']:.2f}%
Average time: {avg_metrics['execution_time']:.2f}s\n"""

    print(summary_log)
    with open(LOG_FILE, "a") as f:
        f.write(summary_log)

# Salvare rezultate detaliate
results_df = pd.DataFrame(all_metrics)
# results_df.to_csv(OUTPUT_FILE, index=False)
# print(f"\n📊 Results saved to: {OUTPUT_FILE}")
print(f"📄 Full log saved to: {LOG_FILE}")


Run 1/5
Run 1 Results:
✅ Correct: 2395 (97.76%)
❌ Wrong: 55 (2.24%)
🎯 Perfect: 468 (95.51%)
⚠️ Mismatch: 22 (4.49%)
📊 Accuracy: 97.76%
⏱️ Time: 12540.67s


Run 2/5
Run 2 Results:
✅ Correct: 2406 (98.20%)
❌ Wrong: 44 (1.80%)
🎯 Perfect: 471 (96.12%)
⚠️ Mismatch: 19 (3.88%)
📊 Accuracy: 98.20%
⏱️ Time: 12261.30s


Run 3/5
Run 3 Results:
✅ Correct: 2410 (98.37%)
❌ Wrong: 40 (1.63%)
🎯 Perfect: 475 (96.94%)
⚠️ Mismatch: 15 (3.06%)
📊 Accuracy: 98.37%
⏱️ Time: 12311.74s


Run 4/5


METODA 3- QML ponderi fixe

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import pennylane as qml
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Configurații
NUM_RUNS = 5  # Numărul de rulări
PREPROCESSED_FILE = "/content/drive/MyDrive/QuantumElearningDataSet/processed_summaries_q_s.csv"
MAIN_DATA_FILE = "/content/drive/MyDrive/QuantumElearningDataSet/arxiv_quantum_computing_pdf.csv"
LOG_FILE = "/content/drive/MyDrive/Quantum-LogFiles/6_quanutum_fidelity_ponderi_fixe_logs.txt"
N_QUBITS = 6  # Numărul de qubiți pentru circuitul cuantic

# Funcție pentru maparea caracteristicilor cuantice
weights = np.random.random(size=(1, 6))
def quantum_feature_map(x):
    n_qubits = len(x)
    dev = qml.device("default.qubit", wires=n_qubits)
    @qml.qnode(dev)
    def circuit():
        for i, feature in enumerate(x):
            qml.RY(feature, wires=i)
        # weights = np.random.random(size=(1, n_qubits))
        qml.templates.BasicEntanglerLayers(weights=weights, wires=range(n_qubits))
        return qml.state()
    return circuit()

# Funcție pentru calculul similarității cuantice
def quantum_similarity(vectors):
    quantum_states = np.array([quantum_feature_map(vec) for vec in vectors])
    density_matrices = qml.math.dm_from_state_vector(quantum_states)  # Vectorizat

    # Calcul fidelitate pairwise folosind broadcasting
    n = len(quantum_states)
    fidelities = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):  # Triunghi superior
            fidelities[i][j] = qml.math.fidelity(density_matrices[i], density_matrices[j])
            fidelities[j][i] = fidelities[i][j]  # Simetrie
    return fidelities

# def quantum_similarity(vectors):
#     quantum_states = [quantum_feature_map(vec) for vec in vectors]
#     fidelities = np.zeros((len(quantum_states), len(quantum_states)))

#     for i in range(len(quantum_states)):
#         for j in range(len(quantum_states)):
#             if i<=j:
#               # Convert the quantum state vectors to density matrices
#               state_i = qml.math.dm_from_state_vector(quantum_states[i])
#               state_j = qml.math.dm_from_state_vector(quantum_states[j])

#               # Calculate the fidelity between the two states
#               fidelity = qml.math.fidelity(state_i, state_j)
#               fidelities[i][j] = fidelity
#               fidelities[j][i] = fidelity
#     return fidelities

# Funcție pentru preprocesarea datelor
def preprocess_data():
    # Încărcare date preprocesate
    df_processed = pd.read_csv(PREPROCESSED_FILE)

    # Convertire text în vectori TF-IDF
    vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(df_processed["Processed_Summary"]).toarray()

    # Reducere dimensionalitate
    svd = TruncatedSVD(n_components=N_QUBITS)
    reduced_matrix = svd.fit_transform(tfidf_matrix)

    return reduced_matrix

# Funcție pentru rularea unei singure iterații
def run_single_iteration(run_id, df, reduced_matrix):
    start_time = time.time()

    # Generare matrice de similaritate cuantică
    similarity_matrix = quantum_similarity(reduced_matrix)

    metrics = {
        'run_id': run_id,
        'nr_diferente': 0,
        'total_corecte': 0,
        'perfect_matches': 0,
        'at_least_one_mismatch': 0,
        'accuracies': []
    }

    for i, row in df.iterrows():
        similarities = similarity_matrix[i]
        top_5_indices = np.argsort(similarities)[-6:-1][::-1]
        current_category = df.iloc[i]["Category"]
        nr_corecte = sum(df.iloc[idx]["Category"] == current_category for idx in top_5_indices)

        metrics['accuracies'].append((nr_corecte / 5) * 100)
        metrics['total_corecte'] += nr_corecte
        metrics['nr_diferente'] += (5 - nr_corecte)

        if nr_corecte == 5:
            metrics['perfect_matches'] += 1
        else:
            metrics['at_least_one_mismatch'] += 1

    # Calcul metrici finale
    total_predicted = len(df) * 5
    metrics.update({
        'correct_percentage': (metrics['total_corecte'] / total_predicted) * 100,
        'different_percentage': (metrics['nr_diferente'] / total_predicted) * 100,
        'perfect_match_percentage': (metrics['perfect_matches'] / len(df)) * 100,
        'mismatch_percentage': (metrics['at_least_one_mismatch'] / len(df)) * 100,
        'mean_accuracy': np.mean(metrics['accuracies']),
        'execution_time': time.time() - start_time
    })

    return metrics

# Pregătire fișier de log
with open(LOG_FILE, "w") as f:
    f.write(f"Quantum Similarity Experiment - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Configuration: {NUM_RUNS} runs, {N_QUBITS} qubits\n\n")

# Încărcare date principale
df_main = pd.read_csv(MAIN_DATA_FILE)
reduced_matrix = preprocess_data()
all_metrics = []

# Rulări multiple
for run in range(1, NUM_RUNS+1):
    print(f"\n{'='*40}\nRun {run}/{NUM_RUNS}\n{'='*40}")

    metrics = run_single_iteration(run, df_main, reduced_matrix)
    all_metrics.append(metrics)

    # Scriere log
    log_entry = f"""Run {run} Results:
✅ Correct: {metrics['total_corecte']} ({metrics['correct_percentage']:.2f}%)
❌ Wrong: {metrics['nr_diferente']} ({metrics['different_percentage']:.2f}%)
🎯 Perfect: {metrics['perfect_matches']} ({metrics['perfect_match_percentage']:.2f}%)
⚠️ Mismatch: {metrics['at_least_one_mismatch']} ({metrics['mismatch_percentage']:.2f}%)
📊 Accuracy: {metrics['mean_accuracy']:.2f}%
⏱️ Time: {metrics['execution_time']:.2f}s\n"""

    print(log_entry)
    with open(LOG_FILE, "a") as f:
        f.write(log_entry)

# Calcul medii
if NUM_RUNS > 1:
    avg_metrics = {
        'correct_percentage': np.mean([m['correct_percentage'] for m in all_metrics]),
        'different_percentage': np.mean([m['different_percentage'] for m in all_metrics]),
        'perfect_match_percentage': np.mean([m['perfect_match_percentage'] for m in all_metrics]),
        'mismatch_percentage': np.mean([m['mismatch_percentage'] for m in all_metrics]),
        'mean_accuracy': np.mean([m['mean_accuracy'] for m in all_metrics]),
        'execution_time': np.mean([m['execution_time'] for m in all_metrics])
    }

    summary_log = f"""\n{'='*40}
FINAL SUMMARY AFTER {NUM_RUNS} RUNS
Average correct: {avg_metrics['correct_percentage']:.2f}%
Average wrong: {avg_metrics['different_percentage']:.2f}%
Average perfect: {avg_metrics['perfect_match_percentage']:.2f}%
Average mismatch: {avg_metrics['mismatch_percentage']:.2f}%
Overall accuracy: {avg_metrics['mean_accuracy']:.2f}%
Average time: {avg_metrics['execution_time']:.2f}s\n"""

    print(summary_log)
    with open(LOG_FILE, "a") as f:
        f.write(summary_log)

# Salvare rezultate detaliate
results_df = pd.DataFrame(all_metrics)
# results_df.to_csv(OUTPUT_FILE, index=False)
# print(f"\n📊 Results saved to: {OUTPUT_FILE}")
print(f"📄 Full log saved to: {LOG_FILE}")


Run 1/5
Run 1 Results:
✅ Correct: 2450 (100.00%)
❌ Wrong: 0 (0.00%)
🎯 Perfect: 490 (100.00%)
⚠️ Mismatch: 0 (0.00%)
📊 Accuracy: 100.00%
⏱️ Time: 398.55s


Run 2/5
Run 2 Results:
✅ Correct: 2450 (100.00%)
❌ Wrong: 0 (0.00%)
🎯 Perfect: 490 (100.00%)
⚠️ Mismatch: 0 (0.00%)
📊 Accuracy: 100.00%
⏱️ Time: 392.08s


Run 3/5
Run 3 Results:
✅ Correct: 2450 (100.00%)
❌ Wrong: 0 (0.00%)
🎯 Perfect: 490 (100.00%)
⚠️ Mismatch: 0 (0.00%)
📊 Accuracy: 100.00%
⏱️ Time: 380.28s


Run 4/5
Run 4 Results:
✅ Correct: 2450 (100.00%)
❌ Wrong: 0 (0.00%)
🎯 Perfect: 490 (100.00%)
⚠️ Mismatch: 0 (0.00%)
📊 Accuracy: 100.00%
⏱️ Time: 377.67s


Run 5/5
Run 5 Results:
✅ Correct: 2450 (100.00%)
❌ Wrong: 0 (0.00%)
🎯 Perfect: 490 (100.00%)
⚠️ Mismatch: 0 (0.00%)
📊 Accuracy: 100.00%
⏱️ Time: 381.08s


FINAL SUMMARY AFTER 5 RUNS
Average correct: 100.00%
Average wrong: 0.00%
Average perfect: 100.00%
Average mismatch: 0.00%
Overall accuracy: 100.00%
Average time: 385.93s

📄 Full log saved to: /content/drive/MyDrive/Quantum-L